In [ ]:
!gdown 1u6uANPoai_N6ytg9zFbT2ISYVUqWtJ_X

Downloading...
From: https://drive.google.com/uc?id=1u6uANPoai_N6ytg9zFbT2ISYVUqWtJ_X
To: /content/train_dataset_Минприроды.zip
100% 1.76G/1.76G [00:15<00:00, 111MB/s]


In [ ]:
%%capture
!unzip /content/train_dataset_Минприроды.zip

In [ ]:
%%capture
!unzip /content/train_dataset_Минприроды/разметка_кликун.zip
!unzip /content/train_dataset_Минприроды/разметка_малый.zip
!unzip /content/train_dataset_Минприроды/разметка_шипун.zip

In [ ]:
!mkdir /content/MODELDATA

In [ ]:
!mv /content/разметка_малый/images /content/MODELDATA/mal
!mv /content/разметка_шипун/images /content/MODELDATA/ship
!mv /content/klikun/images /content/MODELDATA/skikun

In [ ]:
%%capture
!pip install hugsvision
!pip install --upgrade accelerate

In [ ]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
len(os.listdir('/content/MODELDATA/skikun'))

756

In [ ]:
!pip uninstall -y transformers
!pip install transformers==4.28.0

In [ ]:
!rm -rf /content/MODELDATA/ship/masks

In [ ]:
import os
import random

def half(path):
    folder_path = path

    # get a list of all the files in the folder
    file_list = os.listdir(folder_path)

    # shuffle the list to select random files to delete
    random.shuffle(file_list)

    # delete every other file in the shuffled list
    for i in range(0, len(file_list), 2):
        file_path = os.path.join(folder_path, file_list[i])
        os.remove(file_path)

In [ ]:
half('/content/MODELDATA/mal')
half('/content/MODELDATA/mal')
#half('/content/MODELDATA/mal')

half('/content/MODELDATA/ship')
half('/content/MODELDATA/ship')
#half('/content/MODELDATA/ship')

half('/content/MODELDATA/skikun')
half('/content/MODELDATA/skikun')
#half('/content/MODELDATA/skikun')

In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
  "/content/MODELDATA",
  test_ratio   = 0.15,
  balanced     = True,
  augmentation = False,
)

Split Datasets...
Balance train dataset...
The less represented label in train as 750 occurrences
Size of train after balancing is 2250
Training Dataset Elements:  1912
+---------+-----+------+--------+-------+
| Dataset | mal | ship | skikun | Total |
+---------+-----+------+--------+-------+
|  Train  | 621 | 657  |  634   | 1912  |
|  Test   | 129 |  93  |  116   |  338  |
+---------+-----+------+--------+-------+


In [ ]:
huggingface_model = 'google/vit-base-patch16-224-in21k'

In [ ]:
len(train)

1912

In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification

trainer = VisionClassifierTrainer(
	model_name   = "MyKvasirV2Model",
	train        = train,
	test         = test,
	output_dir   = "/content/out/",
	max_epochs   = 10,
	batch_size   = 32,
	lr	     = 1e-5,
	fp16	     = True,

	model = ViTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label
	),

	feature_extractor = ViTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-p

{'0': 'mal', '1': 'ship', '2': 'skikun'}
{'mal': '0', 'ship': '1', 'skikun': '2'}
Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.756546
2,No log,0.533713
3,No log,0.421478
4,No log,0.358522
5,No log,0.322278
6,No log,0.303349
7,No log,0.296473
8,No log,0.288467
9,0.342600,0.286772
10,0.342600,0.286050


Model saved at: /content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48


In [ ]:
ref, hyp = trainer.evaluate_f1_score()

100%|██████████| 338/338 [00:20<00:00, 16.56it/s]

              precision    recall  f1-score   support

         mal     0.9091    0.8527    0.8800       129
        ship     0.9579    0.9785    0.9681        93
      skikun     0.8607    0.9052    0.8824       116

    accuracy                         0.9053       338
   macro avg     0.9092    0.9121    0.9101       338
weighted avg     0.9059    0.9053    0.9050       338

Logs saved at: /content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48


In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ref, hyp)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
plt.savefig("/content/conf_matrix_1.jpg")

In [ ]:
!zip -r /content/HUGSMODEL.zip /content/out/MYKVASIRV2MODEL

  adding: content/out/MYKVASIRV2MODEL/ (stored 0%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/ (stored 0%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/ (stored 0%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/config.json (deflated 48%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/training_args.bin (deflated 48%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/trainer/pytorch_model.bin (deflated 7%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/logs.txt (deflated 56%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/1684643808.8620117/ (stored 0%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/1684643808.8620117/events.out.tfevents.1684643808.73e35d9c1ffc.159.3 (deflated 62%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/feature_extractor/ (stored 0%)
  adding: content/out/MYKVASIRV2MODEL/10_2023-05-21-04-36-48/feature_extractor/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/HUGSMODEL.zip /content/drive/MyDrive/

In [ ]:
/content/out/MYKVASIRV2MODEL/15_2023-05-20-20-13-18/trainer/pytorch_model.bin

In [ ]:
hugs_path = "/content/out/MYKVASIRV2MODEL/15_2023-05-20-20-13-18/trainer"
classifier = VisionClassifierInference(
    feature_extractor = ViTFeatureExtractor.from_pretrained(hugs_path),
    model = ViTForImageClassification.from_pretrained(hugs_path),
)

Model loaded!


In [ ]:
hugs_class = classifier.predict(img_path='/content/lebed_shipun3.jpg')

In [ ]:
hugs_class

'ship'